In [1]:
import keras

m = keras.models.load_model('cifar/models/shadow_models/0.1/train/0.keras')

2024-04-04 19:40:25.457938: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 19:40:25.462678: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 19:40:28.617026: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
m.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ random_flip (RandomFlip)        │ (None, 32, 32, 3)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 32, 32, 32)     │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization             │ (None, 32, 32, 32)     │            64 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 16, 16, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 16, 16, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_1           │ (None, 16, 16, 32)     │            64 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 8, 8, 32)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 8, 8, 64)       │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_2           │ (None, 8, 8, 64)       │           128 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 4, 4, 64)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 512)            │       524,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ group_normalization_3           │ (None, 512)            │         1,024 │
│ (GroupNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 10)             │         5,130 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,679,552 (6.41 MB)

 Trainable params: 559,850 (2.14 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 1,119,702 (4.27 MB)

In [5]:
len(m.layers)

14

In [6]:
dense_1 = m.layers[11]

In [7]:
dense_2 = m.layers[13]

In [10]:
dense_1.weights

[<KerasVariable shape=(1024, 512), dtype=float32, path=dense/kernel>,
 <KerasVariable shape=(512,), dtype=float32, path=dense/bias>]

In [13]:
dense_2.weights

[<KerasVariable shape=(512, 10), dtype=float32, path=dense_1/kernel>,
 <KerasVariable shape=(10,), dtype=float32, path=dense_1/bias>]

In [11]:
conv_1 = m.layers[1]

In [12]:
conv_1.weights

[<KerasVariable shape=(3, 3, 3, 32), dtype=float32, path=conv2d/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=conv2d/bias>]

In [14]:
conv_2 = m.layers[4]
conv_2.weights

[<KerasVariable shape=(3, 3, 32, 32), dtype=float32, path=conv2d_1/kernel>,
 <KerasVariable shape=(32,), dtype=float32, path=conv2d_1/bias>]

In [44]:
def do_read_single_model_params(m):
    layer_indices_for_adv = [
        1, #conv2d
        4, #conv2d_1
        7, #conv2d_2
        11, #dense
        13 #dense_1
    ]
    return [m.layers[i].weights for i in layer_indices_for_adv]

adv_inputs = do_read_single_model_params(m)

In [77]:
def transform_input_name(name):
    return name.replace("/", "-")

In [49]:
def read_single_model_params(path):
    model = keras.models.load_model(path)
    return do_read_single_model_params(model)

In [57]:
import multiprocessing as mp

def read_mult_model_params_mp(paths, nr_cpu):
    #pool = mp.Pool(nr_cpu)
    #w_b = pool.map(read_single_model_params, paths)
    #pool.close()
    
    w_b = [read_single_model_params(p) for p in paths]
    
    return w_b

In [62]:
models_per_y = 2 # 200
all_y = [0.1, 0.2, 0.3, 0.4, 0.5]#, 0.6, 0.7, 0.8, 0.9]]
base_path = "cifar/models/shadow_models"
paths = [f"{base_path}/{j}/train/{i}.keras" for i in range(models_per_y) for j in all_y]

wb = read_mult_model_params_mp(paths, 10)

In [64]:
import numpy as np

y = np.repeat(all_y, models_per_y)
y

array([0.1, 0.1, 0.2, 0.2, 0.3, 0.3, 0.4, 0.4, 0.5, 0.5])

In [92]:
import tensorflow as tf

def get_dataset(wb, y):
    def my_generator():
        for w, my_y in zip(wb, y):
            flattened_w = [item for row in w for item in row]
            yield {transform_input_name(single_weight.path): single_weight for single_weight in flattened_w}, my_y
    
    out_types = {transform_input_name(single_weight.path): tf.float32 for w in wb[1] for single_weight in w}
    return tf.data.Dataset.from_generator(my_generator, output_types=(out_types, tf.int32))

In [90]:
flat_wb = [item for row in wb for item in row]
even_flatter_wb = [item for row in flat_wb for item in row]
flat_wb
wb[1]

[[<KerasVariable shape=(3, 3, 3, 32), dtype=float32, path=conv2d_600/kernel>,
  <KerasVariable shape=(32,), dtype=float32, path=conv2d_600/bias>],
 [<KerasVariable shape=(3, 3, 32, 32), dtype=float32, path=conv2d_601/kernel>,
  <KerasVariable shape=(32,), dtype=float32, path=conv2d_601/bias>],
 [<KerasVariable shape=(3, 3, 32, 64), dtype=float32, path=conv2d_602/kernel>,
  <KerasVariable shape=(64,), dtype=float32, path=conv2d_602/bias>],
 [<KerasVariable shape=(1024, 512), dtype=float32, path=dense_400/kernel>,
  <KerasVariable shape=(512,), dtype=float32, path=dense_400/bias>],
 [<KerasVariable shape=(512, 10), dtype=float32, path=dense_401/kernel>,
  <KerasVariable shape=(10,), dtype=float32, path=dense_401/bias>]]

In [93]:
ds = get_dataset(wb, y)
ds


<_FlatMapDataset element_spec=({'conv2d_600-kernel': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'conv2d_600-bias': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'conv2d_601-kernel': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'conv2d_601-bias': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'conv2d_602-kernel': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'conv2d_602-bias': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'dense_400-kernel': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'dense_400-bias': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'dense_401-kernel': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None), 'dense_401-bias': TensorSpec(shape=<unknown>, dtype=tf.float32, name=None)}, TensorSpec(shape=<unknown>, dtype=tf.int32, name=None))>

conv_1 weights: KerasVariable shape=(3, 3, 3, 32)
3x3 filter für 3 channels, 32 stück
Die Reihenfolge *innerhalb* der filter ist wichtig, aber die Reihenfolge der filter selbst ist nicht relevant.

In [24]:
class ConvSplitter(keras.layers.Layer):
    def __init__(self, neurons):
        super().__init__()
        self.neurons = neurons
    
    #overwrite get_config for being able to save and load model
    def get_config(self):
        config = {
            "neurons" : self.neurons
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
        return [inputs[:, :, :, :, i] for i in range(self.neurons)]

    
    
class WeightsSplitter(keras.layers.Layer):
    def __init__(self, neurons):
        super().__init__()
        self.neurons = neurons
    
    #overwrite get_config for being able to save and load model
    def get_config(self):
        config = {
            "neurons" : self.neurons
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
        return [inputs[:, :, i] for i in range(self.neurons)]

    
    
class BiasSplitter(keras.layers.Layer):
    def __init__(self, neurons):
        super().__init__()
        self.neurons = neurons
        
    #overwrite get_config for being able to save and load model
    def get_config(self):
        config = {
            "neurons" : self.neurons
        }
        base_config = super().get_config()
        return dict(list(base_config.items()) + list(config.items()))

    def call(self, inputs):
        return [inputs[:, i] for i in range(self.neurons)]


In [73]:
from keras import layers

def get_wb_meta_clf(adv_inputs):
    inputs = []
    kernel_splitters = []
    bias_splitters = []
    
    phi_layers = []
    for i in range(len(adv_inputs)):
        for my_input in adv_inputs[i]:
            keras_input = keras.Input(shape=my_input.shape, name=transform_input_name(my_input.path))
            inputs.append(keras_input)
            if my_input.path.find("conv") >= 0 and my_input.path.find("kernel") >= 0:
                kernel_splitters.append(ConvSplitter(my_input.shape[3])(keras_input))
            elif my_input.path.find("bias") >= 0:
                bias_splitters.append(BiasSplitter(my_input.shape[0])(keras_input))
            else:
                kernel_splitters.append(WeightsSplitter(my_input.shape[1])(keras_input))
                
    phi_layers = []
    for kernels, biases in zip(kernel_splitters, bias_splitters):
        channel_outputs = []
        for w, b in zip(kernels, biases):
            flatten_w = layers.Flatten()(w)
            reshaped_b = layers.Reshape((1,))(b)
            concat = layers.concatenate([flatten_w, reshaped_b])
            phi = layers.Dense(10, activation="relu")(concat)
            # ? phi = layers.Dropout(0.5)(phi)
            channel_outputs.append(phi)
        my_phi = layers.Add()(channel_outputs)
        phi_layers.append(my_phi)
    
    x = layers.BatchNormalization()(layers.concatenate(phi_layers))
    x = layers.Dense(30, activation='relu')(x)
    x = layers.Dense(8, activation='relu')(x)
    output = layers.Dense(1)(x)
    
    return keras.Model(inputs=inputs, outputs=output)
    
    # manual_adversary.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.R2Score()])
    


In [39]:
meta_clf = get_wb_meta_clf(adv_inputs)

In [42]:
#

In [41]:
meta_clf.compile(optimizer=keras.optimizers.Adam(), loss=keras.losses.MeanSquaredError(), metrics=[keras.metrics.R2Score()])